In [ ]:
!pip install sagemaker==1.72.0

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer
import boto3
import json
import pickle
from datetime import datetime

In [9]:
AWS_REGION = 'us-east-1'
IAM_CLIENT = boto3.client('iam',region_name=AWS_REGION)
LAMBDA_CLIENT = boto3.client('lambda',region_name=AWS_REGION)
API_CLIENT = boto3.client('apigateway',region_name=AWS_REGION)
S3_CLIENT = boto3.client('s3',region_name=AWS_REGION)
RUNTIME_CLIENT = boto3.client('sagemaker-runtime',region_name=AWS_REGION)
APP_NAME = f'sentiment-analysis-web-app-{datetime.now().strftime("%d-%m-%Y-%H-%M-%S")}'

In [10]:
APP_NAME = 'sentiment-analysis-web-app-08-02-2022-18-17-59'

## Preparing and Processing the data

In [2]:
data = {}
labels = {}

for data_type in ['train', 'test']:
    data[data_type] = {}
    labels[data_type] = {}
    
    for sentiment in ['pos', 'neg']:
        data[data_type][sentiment] = []
        labels[data_type][sentiment] = []
        
        files = glob.glob(f'data/{data_type}/{sentiment}/*.txt')
        
        for f in files:
            with open(f,encoding='utf8') as review:
                data[data_type][sentiment].append(review.read())
                # Here we represent a positive review by '1' and a negative review by '0'
                labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)

In [7]:
train = pd.DataFrame({'text':data['train']['pos'] + data['train']['neg'],'sentiment':labels['train']['pos'] + labels['train']['neg']})
test = pd.DataFrame({'text':data['test']['pos'] + data['test']['neg'],'sentiment':labels['test']['pos'] + labels['test']['neg']})

In [8]:
train['sentiment'] = train['sentiment'].replace({1:'pos',0:'neg'})
test['sentiment'] = test['sentiment'].replace({1:'pos',0:'neg'})

In [15]:
with pd.ExcelWriter('data/sentiment_data.xlsx', engine='xlsxwriter') as writer:
    test.to_excel(writer,sheet_name='test',index=False,encoding='utf-8')
    train.to_excel(writer,sheet_name='train',index=False,encoding='utf-8')

In [16]:
pd.read_excel('data/sentiment_data.xlsx',sheet_name='test')

,text,sentiment
0,I went and saw this movie last night after bei...,pos
1,Actor turned director Bill Paxton follows up h...,pos
2,As a recreational golfer with some knowledge o...,pos
3,"I saw this film in a sneak preview, and it is ...",pos
4,Bill Paxton has taken the true story of the 19...,pos
...,...,...
24995,I occasionally let my kids watch this garbage ...,neg
24996,When all we have anymore is pretty much realit...,neg
24997,The basic genre is a thriller intercut with an...,neg
24998,Four things intrigued me as to this film - fir...,neg


In [19]:
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [20]:
train_x, train_y = shuffle(data['train']['pos'] + data['train']['neg'], labels['train']['pos'] + labels['train']['neg'])
test_x, test_y = shuffle(data['test']['pos'] + data['test']['neg'], labels['test']['pos'] + labels['test']['neg'])

In [21]:
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_x), len(test_x)))

IMDb reviews (combined): train = 25000, test = 25000


In [22]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [23]:
train_x = [review_to_words(review) for review in train_x]
test_x = [review_to_words(review) for review in test_x]

Fit a vectorizer to training documents and use it to transform them

Apply the same vectorizer to transform the test documents (ignore unknown words)

NOTE: Training documents have already been preprocessed and tokenized into words; pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x

In [24]:
vectorizer = CountVectorizer(max_features=5000)
train_x = vectorizer.fit_transform(train_x).toarray()
test_x = vectorizer.transform(test_x).toarray()
vocabulary = vectorizer.vocabulary_

In [26]:
pd.DataFrame(test_x).to_csv('data/test.csv', header=False, index=False)
pd.concat([pd.DataFrame(train_y[:10000]),pd.DataFrame(train_x[:10000])], axis=1).to_csv('data/validation.csv', header=False, index=False)
pd.concat([pd.DataFrame(train_y[10000:]),pd.DataFrame(train_x[10000:])], axis=1).to_csv('data/train.csv', header=False, index=False)

In [53]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.intc):
            return int(obj)

with open('data/vocabulary.json', 'w',encoding='utf8') as fp:
    json.dump(vectorizer.vocabulary_, fp, indent=4,cls=NumpyEncoder)

In [27]:
[bucket['Name'] for bucket in S3_CLIENT.list_buckets()['Buckets'] if S3_CLIENT.get_bucket_tagging(Bucket=bucket['Name'])['TagSet'][0]['Value'] == 'sentiment-analysis-web-app']

['sentiment-analysis-web-app-08-02-2022-18-17-59']

## Upload data to S3

In [4]:
bucket = S3_CLIENT.create_bucket(Bucket=APP_NAME + '-bucket')
bucket

{'ResponseMetadata': {'RequestId': 'GV6XCTWHG9HY98C5',
  'HostId': '6Kt7c2bj4ffApa9sUnEyercJVotOHDuwHhsjEDvkskAw2h5bUN3nKwkpuDRGGI621aQic3GcG4k=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '6Kt7c2bj4ffApa9sUnEyercJVotOHDuwHhsjEDvkskAw2h5bUN3nKwkpuDRGGI621aQic3GcG4k=',
   'x-amz-request-id': 'GV6XCTWHG9HY98C5',
   'date': 'Tue, 08 Feb 2022 16:20:43 GMT',
   'location': '/sentiment-analysis-web-app-08-02-2022-18-17-59',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/sentiment-analysis-web-app-08-02-2022-18-17-59'}

In [6]:
S3_CLIENT.put_bucket_tagging(
    Bucket=APP_NAME + '-bucket',
    Tagging={
        'TagSet': [
            {
                'Key': 'Name',
                'Value': 'sentiment-analysis-web-app'
            },
        ]
    },
)

{'ResponseMetadata': {'RequestId': 'AJVHKTKM6BCXWXE8',
  'HostId': 'HaPhDSAUtoXzhlpVAqgsUJyHW8acaF+dBDauxfw8SeJM42EG3m+QM4V4nc8HlzuFXt5B69WoMC0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'HaPhDSAUtoXzhlpVAqgsUJyHW8acaF+dBDauxfw8SeJM42EG3m+QM4V4nc8HlzuFXt5B69WoMC0=',
   'x-amz-request-id': 'AJVHKTKM6BCXWXE8',
   'date': 'Tue, 08 Feb 2022 16:21:26 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [29]:
S3_CLIENT.upload_file(Filename = 'data/test.csv', Bucket =APP_NAME + '-bucket', Key = 'data/test.csv')

In [30]:
S3_CLIENT.upload_file(Filename = 'data/validation.csv', Bucket =APP_NAME + '-bucket', Key = 'data/validation.csv')

In [31]:
S3_CLIENT.upload_file(Filename = 'data/train.csv', Bucket =APP_NAME + '-bucket', Key = 'data/train.csv')

In [54]:
S3_CLIENT.upload_file(Filename = 'data/vocabulary.json', Bucket =APP_NAME + '-bucket', Key = 'data/vocabulary.json')

## Creating the XGBoost model

In [9]:
session = sagemaker.Session()

In [7]:
AssumeRolePolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

sagemaker_role = IAM_CLIENT.create_role(
                            RoleName=APP_NAME + '-sagemaker',
                            AssumeRolePolicyDocument=json.dumps(AssumeRolePolicy),
                          )
sagemaker_role['Role']['RoleName']

'sentiment-analysis-web-app-08-02-2022-18-17-59-sagemaker'

In [21]:
InlinePolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Effect": "Allow",
            "Resource": [
                f"arn:aws:s3:::{APP_NAME + '-bucket'}"
            ]
        },
        {
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject"
            ],
            "Effect": "Allow",
            "Resource": [
                f"arn:aws:s3:::{APP_NAME + '-bucket'}/*"
            ]
        }
    ]
}
IAM_CLIENT.put_role_policy(
                            RoleName=sagemaker_role['Role']['RoleName'],
                            PolicyName='AmazonSageMaker-ExecutionPolicy',
                            PolicyDocument=json.dumps(InlinePolicy)
                            )

{'ResponseMetadata': {'RequestId': 'd89169ef-f0a4-4fef-83fd-1de55b08406b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd89169ef-f0a4-4fef-83fd-1de55b08406b',
   'content-type': 'text/xml',
   'content-length': '206',
   'date': 'Wed, 09 Feb 2022 16:37:56 GMT'},
  'RetryAttempts': 0}}

In [8]:
IAM_CLIENT.attach_role_policy(
                                RoleName=sagemaker_role['Role']['RoleName'],
                                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
                            )

{'ResponseMetadata': {'RequestId': '7d1d758e-7d8e-4ed9-a7e9-e34db9f6f059',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7d1d758e-7d8e-4ed9-a7e9-e34db9f6f059',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 09 Feb 2022 15:40:53 GMT'},
  'RetryAttempts': 0}}

In [22]:
container = get_image_uri(session.boto_region_name, 'xgboost','1.0-1')
container

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

In [23]:
xgb = sagemaker.estimator.Estimator(image_name=container, # The location of the container we wish to use
                                    role=sagemaker_role['Role']['Arn'],                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path=f"s3://{APP_NAME + '-bucket'}/output",
                                    sagemaker_session=session)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [24]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

In [25]:
s3_input_train = sagemaker.s3_input(s3_data='s3://sentiment-analysis-web-app-08-02-2022-18-17-59/data/train.csv', content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://sentiment-analysis-web-app-08-02-2022-18-17-59/data/validation.csv', content_type='csv')
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2022-02-09 16:40:18 Starting - Starting the training job...
2022-02-09 16:40:20 Starting - Launching requested ML instances......
2022-02-09 16:41:48 Starting - Preparing the instances for training......
2022-02-09 16:43:03 Downloading - Downloading input data...
2022-02-09 16:43:20 Training - Downloading the training image.........
2022-02-09 16:45:17 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[16:45:23] 15000x5000 matrix with 75000000 entries loaded 

## Deploying the model

In [33]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, 
                        instance_type = 'ml.m4.xlarge',
                        serializer=sagemaker.predictor.csv_serializer,
                        content_type='text/csv',
                        endpoint_name=APP_NAME + '-endpoint',
                        model_name=APP_NAME + '-model')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: sagemaker-xgboost-2022-02-09-16-40-19-453


---------!

In [6]:
test_x = pd.read_csv('data/test.csv',header=None).values

In [51]:
predictions = ''
for data in np.array_split(test_x, 500):
    predictions += (',' + xgb_predictor.predict(data).decode('utf-8'))

In [59]:
predictions = np.round(np.array(predictions.split(','))[1:].astype(float))
predictions

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
accuracy_score(test_y, predictions)

In [ ]:
','.join([str(val) for val in test_x[0]]).encode('utf-8')

In [16]:
response = RUNTIME_CLIENT.invoke_endpoint(EndpointName = APP_NAME + '-endpoint',# The name of the endpoint we created
                                    ContentType = 'text/csv',
                                    Body = ','.join([str(val) for val in test_x[1]]).encode('utf-8'))
round(float(response['Body'].read().decode('utf-8')))

1

In [ ]:
xgb_predictor.delete_endpoint()

## Create an IAM Role for the Lambda function

In [3]:
AssumeRolePolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sts:AssumeRole"
            ],
            "Principal": {
                "Service": [
                    "lambda.amazonaws.com"
                ]
            }
        }
    ]
}

role = IAM_CLIENT.create_role(
                            RoleName='Lambda_SageMaker_Role',
                            AssumeRolePolicyDocument=json.dumps(AssumeRolePolicy),
                          )
role['Role']['RoleName']

{'Role': {'Path': '/',
  'RoleName': 'Lambda_SageMaker_Role',
  'RoleId': 'AROAYUHWLEEAXBIVWUPP6',
  'Arn': 'arn:aws:iam::593222115585:role/Lambda_SageMaker_Role',
  'CreateDate': datetime.datetime(2022, 2, 8, 11, 56, 26, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Action': ['sts:AssumeRole'],
     'Principal': {'Service': ['lambda.amazonaws.com']}}]}},
 'ResponseMetadata': {'RequestId': 'b8843768-c70d-4581-a815-927f27eebb13',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b8843768-c70d-4581-a815-927f27eebb13',
   'content-type': 'text/xml',
   'content-length': '808',
   'date': 'Tue, 08 Feb 2022 11:56:26 GMT'},
  'RetryAttempts': 0}}

In [4]:
IAM_CLIENT.attach_role_policy(
                                RoleName=role['Role']['RoleName'],
                                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
                            )

{'ResponseMetadata': {'RequestId': '39201e74-c01c-4ff3-bfa0-22c4fe1f257b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '39201e74-c01c-4ff3-bfa0-22c4fe1f257b',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Tue, 08 Feb 2022 11:56:30 GMT'},
  'RetryAttempts': 0}}

## Create a Lambda function

In [6]:
with open('lambda.zip', 'rb') as f:
	zipped_code = f.read()

response = LAMBDA_CLIENT.create_function(
                                        FunctionName='sentiment_analysis_xgboost_func',
                                        Runtime='python3.6',
                                        Role=role['Role']['Arn'],
                                        Handler='handler.lambda_handler',
                                        Code={'ZipFile':zipped_code},
                                        Timeout=300, # Maximum allowable timeout
                                    )

response

{'ResponseMetadata': {'RequestId': '644853ea-5ab0-4933-9aa2-926b612eb6c9',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 11:57:18 GMT',
   'content-type': 'application/json',
   'content-length': '987',
   'connection': 'keep-alive',
   'x-amzn-requestid': '644853ea-5ab0-4933-9aa2-926b612eb6c9'},
  'RetryAttempts': 0},
 'FunctionName': 'sentiment_analysis_xgboost_func',
 'FunctionArn': 'arn:aws:lambda:us-east-1:593222115585:function:sentiment_analysis_xgboost_func',
 'Runtime': 'python3.6',
 'Role': 'arn:aws:iam::593222115585:role/Lambda_SageMaker_Role',
 'Handler': 'handler.lambda_handler',
 'CodeSize': 1162,
 'Description': '',
 'Timeout': 300,
 'MemorySize': 128,
 'LastModified': '2022-02-08T11:57:18.139+0000',
 'CodeSha256': 'YDKhEIzzPUrKvaUCiys38siaVqS5qFziXXiwKYrxCXA=',
 'Version': '$LATEST',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '4b1921a1-7642-4884-a15a-4cba5b5936e9',
 'State': 'Pending',
 'StateReason': 'The function is being created.'

## Setting up API Gateway

In [40]:
api = API_CLIENT.create_rest_api(
                                name='sentiment_analysis_web_app',
                                description='sentiment_analysis_web_app_description',
                                endpointConfiguration={'types': ['REGIONAL']}
                                )
api

{'ResponseMetadata': {'RequestId': '4394d05b-8072-40e3-9bf1-9470e9c659b2',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:00 GMT',
   'content-type': 'application/json',
   'content-length': '254',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4394d05b-8072-40e3-9bf1-9470e9c659b2',
   'x-amz-apigw-id': 'NOVbNLcJIAMEcdw='},
  'RetryAttempts': 0},
 'id': 'hcz3k9vcc5',
 'name': 'sentiment_analysis_web_app',
 'description': 'sentiment_analysis_web_app_description',
 'createdDate': datetime.datetime(2022, 2, 8, 14, 59, tzinfo=tzlocal()),
 'apiKeySource': 'HEADER',
 'endpointConfiguration': {'types': ['REGIONAL']},
 'disableExecuteApiEndpoint': False}

## Create 'POST' Method

In [41]:
api_resources = API_CLIENT.get_resources(restApiId=api["id"])
api_resources

{'ResponseMetadata': {'RequestId': 'e79b1e12-23b1-4b0e-b16c-f0e6c553d884',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:05 GMT',
   'content-type': 'application/json',
   'content-length': '42',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e79b1e12-23b1-4b0e-b16c-f0e6c553d884',
   'x-amz-apigw-id': 'NOVcAIL9oAMEdog='},
  'RetryAttempts': 0},
 'items': [{'id': 'many8ls9ea', 'path': '/'}]}

In [42]:
api_method = API_CLIENT.put_method(
    restApiId=api["id"],
    resourceId=api_resources['items'][0]['id'],
    httpMethod='POST',
    authorizationType='NONE'
  )
api_method

{'ResponseMetadata': {'RequestId': '818c6705-85fc-478d-9e95-322587c48064',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:10 GMT',
   'content-type': 'application/json',
   'content-length': '72',
   'connection': 'keep-alive',
   'x-amzn-requestid': '818c6705-85fc-478d-9e95-322587c48064',
   'x-amz-apigw-id': 'NOVcwKG6IAMEcuQ='},
  'RetryAttempts': 0},
 'httpMethod': 'POST',
 'authorizationType': 'NONE',
 'apiKeyRequired': False}

In [43]:
put_method_res = API_CLIENT.put_method_response(
                                                restApiId=api["id"],
                                                resourceId=api_resources['items'][0]['id'],
                                                httpMethod='POST',
                                                statusCode='200',
                                                responseModels = {'application/json': 'Empty'}
                                            )
put_method_res

{'ResponseMetadata': {'RequestId': '3e4d4d6a-c957-49af-b3c2-592478fab48c',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:16 GMT',
   'content-type': 'application/json',
   'content-length': '67',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3e4d4d6a-c957-49af-b3c2-592478fab48c',
   'x-amz-apigw-id': 'NOVdsICuoAMEd1g='},
  'RetryAttempts': 0},
 'statusCode': '200',
 'responseModels': {'application/json': 'Empty'}}

## Create integration point

In [44]:
integration = API_CLIENT.put_integration(
                                    restApiId=api["id"],
                                    resourceId=api_resources['items'][0]['id'],
                                    httpMethod='POST',
                                    type='AWS_PROXY', # also referred to as the 'Lambda proxy integration'
                                    integrationHttpMethod='POST',
                                    contentHandling = 'CONVERT_TO_TEXT',
                                    passthroughBehavior='WHEN_NO_MATCH',
                                    uri=f"arn:aws:apigateway:{AWS_REGION}:lambda:path/2015-03-31/functions/{response['FunctionArn']}/invocations",
                                  )

In [45]:
integration

{'ResponseMetadata': {'RequestId': '69ac8bd4-9b77-4ab8-a8d5-9fe904b43f79',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:21 GMT',
   'content-type': 'application/json',
   'content-length': '354',
   'connection': 'keep-alive',
   'x-amzn-requestid': '69ac8bd4-9b77-4ab8-a8d5-9fe904b43f79',
   'x-amz-apigw-id': 'NOVeiLkXIAMEcdw='},
  'RetryAttempts': 0},
 'type': 'AWS_PROXY',
 'httpMethod': 'POST',
 'uri': 'arn:aws:apigateway:us-east-1:lambda:path/2015-03-31/functions/arn:aws:lambda:us-east-1:593222115585:function:sentiment_analysis_xgboost_func/invocations',
 'passthroughBehavior': 'WHEN_NO_MATCH',
 'contentHandling': 'CONVERT_TO_TEXT',
 'timeoutInMillis': 29000,
 'cacheNamespace': 'many8ls9ea',
 'cacheKeyParameters': []}

In [46]:
integration_response = API_CLIENT.put_integration_response(
                                        restApiId=api["id"],
                                        resourceId=api_resources['items'][0]['id'],
                                        httpMethod='POST',
                                        statusCode='200',
                                        responseTemplates= {'application/json': 'None'}
                                    )
integration_response

{'ResponseMetadata': {'RequestId': '75c3630c-5371-4daf-b92b-5fa176a54479',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 12:59:38 GMT',
   'content-type': 'application/json',
   'content-length': '69',
   'connection': 'keep-alive',
   'x-amzn-requestid': '75c3630c-5371-4daf-b92b-5fa176a54479',
   'x-amz-apigw-id': 'NOVhJIj3IAMEdWg='},
  'RetryAttempts': 0},
 'statusCode': '200',
 'responseTemplates': {'application/json': 'None'}}

## Deploy API Gateway

In [56]:
deployment = API_CLIENT.create_deployment(
                                          restApiId=api["id"],
                                          stageName='prod',
                                            )
deployment

{'ResponseMetadata': {'RequestId': 'b53a6c14-7fce-4e43-9b45-62448736f524',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 08 Feb 2022 13:09:13 GMT',
   'content-type': 'application/json',
   'content-length': '41',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b53a6c14-7fce-4e43-9b45-62448736f524',
   'x-amz-apigw-id': 'NOW7AJAMIAMEd_A='},
  'RetryAttempts': 0},
 'id': 'o9qs5y',
 'createdDate': datetime.datetime(2022, 2, 8, 15, 9, 13, tzinfo=tzlocal())}

In [59]:
stage = API_CLIENT.get_stages(
                                restApiId=api['id'],
                                deploymentId=deployment['id']
                            )

In [58]:
f"https://{api['id']}.execute-api.{AWS_REGION}.amazonaws.com/{stage['item'][0]['stageName']}/"

'https://hcz3k9vcc5.execute-api.us-east-1.amazonaws.com/prod/'

---

API_CLIENT.get_resources(restApiId='hcz3k9vcc5')

API_CLIENT.get_integration(
    restApiId='hcz3k9vcc5',
    resourceId='many8ls9ea',
    httpMethod='POST'
)

API_CLIENT.get_integration(
    restApiId='hx40p3ml7i',
    resourceId='56set2yag6',
    httpMethod='POST'
)

API_CLIENT.get_method(
    restApiId='hcz3k9vcc5',
    resourceId='many8ls9ea',
    httpMethod='POST'
)

API_CLIENT.get_method(
    restApiId='hx40p3ml7i',
    resourceId='56set2yag6',
    httpMethod='POST'
)

API_CLIENT.get_integration_response(
    restApiId='hcz3k9vcc5',
    resourceId='many8ls9ea',
    httpMethod='POST',
    statusCode='200'
)

API_CLIENT.get_integration_response(
                                        restApiId='hx40p3ml7i',
                                        resourceId='56set2yag6',
                                        httpMethod='POST',
                                        statusCode='200'
)

API_CLIENT.get_method_response(
    restApiId='hcz3k9vcc5',
    resourceId='many8ls9ea',
    httpMethod='POST',
    statusCode='200'
)

API_CLIENT.get_method_response(
                                        restApiId='hx40p3ml7i',
                                        resourceId='56set2yag6',
                                        httpMethod='POST',
                                        statusCode='200'
)